# Features and labels into plots utilized for the HINTS manuscript.

## Plots 
**Are always healthy vs x (label/symptom, see below)**
| Condition | Label |
| --------- | ----- |
| Healthy | 0 |
| &nbsp; | &nbsp; |
| Skew | 1 |
| &nbsp; | &nbsp; |
| Saccades | 2 |
| &nbsp; | &nbsp; |
| Nystagmus (left) | 3 |
| Nystagmus (right) | 4 |
| Nystagmus (downbeating) | 5 |

## order
1. Imports & folder creation
<br><br>

2. Skew vs Healthy
- Parts of the HINTS exam: 'nase' and 'still'. <br>
*(i.e., look at the nose of the examiner and looking straight ahead.)* 
- For 'camera' and 'world' perspective.<br>
*(i.e., the reference frame of the measurements.)*
<br><br>

3. Saccades
- Parts of the HINTS exam: 'blingHeadTest'. <br>
*(i.e., During the HIT impulses, which usually start 10 seconds after the bling sound indicating start of this measurement.)* 
- For 'world' perspective.<br>
*(i.e., the reference frame of the measurements.)*
<br><br>

4. Nystagmus left & right
- Parts of the HINTS exam: 'nase', 'links', and 'rechts'. <br>
*(i.e., look at the nose of the examiner and looking left/right of the examiner.)* 
- For 'camera' perspective.<br>
*(i.e., the reference frame of the measurements.)*
- Note that we can simply switch the label number in the code to select the other folder/case.  
<br><br>

5. Nystagmus right
- See 4, change label 3 -> 4.
<br><br>

5. Nystagmus downbeating
- Parts of the HINTS exam: 'nase', 'still'. <br>
*(i.e., look at the nose of the examiner and looking straight ahead.)* 
- For 'camera' perspective.<br>
*(i.e., the reference frame of the measurements.)*
<br><br>


In [4]:
# Enable inline plotting for Jupyter notebook
%matplotlib inline

# Standard library imports
import os
# Third-party imports
import pandas as pd
import plotly.graph_objects as go

# Local application imports
from data import *
from utils import *

In [5]:
# Define directories for saving analysis images
DIRS = [
    "../images",
    "../images/skew-vs-healthy",
    "../images/saccades-vs-healthy",
    "../images/nystagmusLeft-vs-healthy",
    "../images/nystagmusDownbeating-vs-healthy"
]

# Create directories if they don't exist
for dir in DIRS:
    if not os.path.exists(dir):
        os.mkdir(dir)

# Skew vs healthy, 1 vs 0
- Labels are: {'Healthy': 0, 'Skew': 1, 'Saccades': 2, 'NystagmusLeft': 3, 'NystagmusRight': 4, 'NystagmusDownbeating': 5}

In [ ]:
"""
Analysis script for HINTS examination data comparing healthy subjects vs patients with conditions (SKEW).
Focuses on eye movement analysis and visualization of vertical positions.
"""

# Configuration settings for data analysis
DATA_RECORDING_SESSIONS = ["nase"]  # Type of recording session
PERSPECTIVES = ["camera"]           # Measurement reference frame
TIME_RANGE = [5, 15]               # Time window for analysis in seconds
RAW = True                         # Use raw direction and position data
SHOW_PLOT = False                  # Control real-time plot display

# Suppress pandas chained assignment warnings
pd.options.mode.chained_assignment = None

# Iterate through recording sessions and perspectives
for DATA_RECORDING_SESSION in DATA_RECORDING_SESSIONS:
    for PERSPECTIVE in PERSPECTIVES:
        # Load and preprocess data
        data_dir, labels_dir = data_path(DATA_RECORDING_SESSION)
        _, camera_features, camera_labels = generate_data(
            data_dir, 
            labels_dir, 
            sort=True, 
            perspective=PERSPECTIVE, 
            raw=RAW, 
            time_range=TIME_RANGE
        )

        # Define columns to exclude from analysis
        dropped_cols = ['stamp', 'delta', 'Direction', 'head', 'Combined']
        figs_healthy = []
        figs_unhealthy = []

        # Process each patient's data
        for idx in range(len(camera_features)):
            PATIENT_NUMBER = idx

            # Only process healthy (0) and skew (1) cases
            if not (camera_labels[PATIENT_NUMBER] in [0, 1]):
                continue

            # Clean and prepare data
            df = camera_features[PATIENT_NUMBER]
            for col in dropped_cols:
                df = colsToDrop(df, col)
            
            # Create new dataframe for analysis
            df_new = pd.DataFrame()

            # Extract relevant measurements based on perspective
            if PERSPECTIVE == "camera":
                # Convert eye positions to millimeters (×1000)
                df_new["y_left"] = abs(df['cameraLeftEyePosition_y']) * 1000
                df_new["y_right"] = abs(df['cameraRightEyePosition_y']) * 1000

            if PERSPECTIVE == "world":
                # Calculate position differences in world coordinates
                df_new['x'] = (abs(df['worldLeftEyePosition_x']) - abs(df['worldRightEyePosition_x'])) * 1000
                df_new['y'] = (abs(df['worldLeftEyePosition_y']) - abs(df['worldRightEyePosition_y'])) * 1000
                df_new['z'] = (abs(df['worldLeftEyePosition_z']) - abs(df['worldRightEyePosition_z'])) * 1000
            df_new['time'] = df['time']

            # Generate plot for current patient
            df = df_new
            fig = plot_sample(df, camera_labels[PATIENT_NUMBER], PATIENT_NUMBER, 
                            DATA_RECORDING_SESSION, SHOW_PLOT)

            # Update trace names for clarity
            name_mapping = {
                'y_left': 'Left eye',
                'y_right': 'Right eye',
            }
            for trace in fig.data:
                if trace.name in name_mapping:
                    trace.name = name_mapping[trace.name]

            # Configure plot axes
            fig.update_yaxes(range=[0, 10], title_text='Vertical eye position [mm]')
            fig.update_xaxes(title_text='Time [seconds]')

            # Define common layout settings
            layout_settings = {
                'title_text': ' ',
                'autosize': False,
                'width': 1000,
                'height': 500,
                'font': dict(size=20, family="Times New Roman"),
                'legend_title': "",
                'margin': dict(l=100, r=50, b=100, t=100, pad=4),
                'xaxis': dict(automargin=True, tickangle=45),
                'yaxis': dict(automargin=True),
                'legend': dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1
                )
            }

            # Set title and store figure based on patient type
            if camera_labels[PATIENT_NUMBER] == 0:
                title = 'Test of Skew for a healthy subject'
                fig.update_layout(**layout_settings)
                figs_healthy.append(fig)
            elif camera_labels[PATIENT_NUMBER] == 1:
                title = 'Test of Skew for a patient with stroke'
                fig.update_layout(**layout_settings)
                figs_unhealthy.append(fig)

            # Set time range and display plot
            fig.update_xaxes(range=TIME_RANGE, title_text='Time [seconds]')
            fig.show()
            
            # Save interactive plot
            fig.write_html(f"{DIRS[1]}/{title}.html")
            # If you want higher resolution as we had in the manuscript
            # fig.write_image(DIRS[1] + "/" + title + ".svg", engine="kaleido", scale=3) 

# Saccades vs healthy, 2 vs 0
- Labels are: {'Healthy': 0, 'Skew': 1, 'Saccades': 2, 'NystagmusLeft': 3, 'NystagmusRight': 4, 'NystagmusDownbeating': 5}

In [ ]:
"""
Analysis script for Head Impulse Test (HIT) data comparing healthy subjects vs patients with saccades.
Focuses on eye and head velocity analysis.
"""

# Configuration settings for data analysis
DATA_RECORDING_SESSION = "blingHeadTest"
PERSPECTIVES = ["world"]    # World perspective preferred due to fast camera movements
TIME_RANGE = [10, 30]       # Time window for analysis in seconds
RAW = True                 # Use raw direction and position data
SHOW_PLOT = False          # Control real-time plot display

# Suppress pandas chained assignment warnings
pd.options.mode.chained_assignment = None

# Load and preprocess data
data_dir, labels_dir = data_path(DATA_RECORDING_SESSION)

for PERSPECTIVE in PERSPECTIVES:
    # Generate feature and label datasets
    _, camera_features, camera_labels = generate_data(
        data_dir, 
        labels_dir, 
        sort=True, 
        perspective=PERSPECTIVE, 
        raw=RAW, 
        time_range=TIME_RANGE
    )

    # Define columns to exclude from analysis
    dropped_cols = ['_z', 'Direction', 'delta_time', 'timestamp', 'Combined', 'Quaternion']
    figs_healthy = []
    figs_unhealthy = []

    # Process each patient's data
    for idx in range(len(camera_features)):
        PATIENT_NUMBER = idx

        # Only process healthy (0) and saccades (2) cases
        if not (camera_labels[PATIENT_NUMBER] in [0, 2]):
            continue

        # Clean and prepare data
        df = camera_features[PATIENT_NUMBER]
        for col in dropped_cols:
            df = colsToDrop(df, col)

        # Calculate velocities based on perspective
        df_new = pd.DataFrame()
        
        if PERSPECTIVE == 'camera':
            df['left_eye_velocity_x'] = df['cameraLeftEyePosition_x'].diff() / df['time'].diff()
            df['right_eye_velocity_x'] = df['cameraRightEyePosition_x'].diff() / df['time'].diff()
        
        if PERSPECTIVE == 'world':
            df['left_eye_velocity_x'] = df['worldLeftEyePosition_x'].diff() / df['time'].diff()
            df['right_eye_velocity_x'] = df['worldRightEyePosition_x'].diff() / df['time'].diff()
        
        # Calculate combined eye velocity and head rotation
        df_new['Eye_velocity_x'] = ((df['left_eye_velocity_x']) + (df['right_eye_velocity_x'])) * 5_000
        df_new['Velocity_Euler_y'] = ((df['headEulerAngles_y'].diff()) / df['time'].diff()) / 5

        # Center velocities around zero
        df_new['Velocity_Euler_y'] -= df_new['Velocity_Euler_y'].mean()
        df_new['Eye_velocity_x'] -= df_new['Eye_velocity_x'].mean()
        df_new['time'] = df['time']

        # Remove NaN values
        df = df_new.dropna()

        # Generate plot
        fig = plot_sample(df, camera_labels[PATIENT_NUMBER], PATIENT_NUMBER, 
                         DATA_RECORDING_SESSION, SHOW_PLOT)

        # Update trace names for clarity
        name_mapping = {
            'Eye_velocity_x': 'Eye velocity',
            'Velocity_Euler_y': 'Head velocity '
        }
        for trace in fig.data:
            if trace.name in name_mapping:
                trace.name = name_mapping[trace.name]

        # Configure plot axes
        fig.update_yaxes(title_text='Velocity [AU/s]  [based on position vector]')
        fig.update_xaxes(title_text='Time [seconds]')

        # Set title based on patient type
        if camera_labels[PATIENT_NUMBER] == 0:
            title = 'Head Impulse Test for a healthy subject'
            figs_healthy.append(fig)
        elif camera_labels[PATIENT_NUMBER] == 2:
            title = 'Head Impulse Test for a patient with saccades'
            figs_unhealthy.append(fig)

        # Configure plot layout
        fig.update_layout(
            title_text=' ',
            autosize=False,
            width=1000,
            height=500,
            font=dict(size=20),
            font_family="Times New Roman",
            legend_title="",
            xaxis={'automargin': True},
            yaxis={'automargin': True}
        )

        # Set time range
        fig.update_xaxes(range=TIME_RANGE, title_text='Time [seconds]')

        # Display plot
        fig.show()

        # Save plot (fast interactive version)
        fig.write_html(f"{DIRS[2]}/{title}.html")
        
        # High resolution version with kaleido (commented out due to long processing time)
        # fig.write_image(f"{DIRS[2]}/{title}.svg", engine="kaleido", scale=3)

# Nystagmus (LEFT) vs healthy, 3 vs 0
- Labels are: {'Healthy': 0, 'Skew': 1, 'Saccades': 2, 'NystagmusLeft': 3, 'NystagmusRight': 4, 'NystagmusDownbeating': 5}

In [ ]:
"""
Analysis script for Nystagmus detection comparing healthy subjects vs patients with left vestibular neuritis.
Analyzes eye movement direction in different gaze positions (straight, left, right).
"""

# Configuration settings for data analysis
DATA_RECORDING_SESSIONS = ["nase", "links", "rechts"]  # Straight, left, right gaze
PERSPECTIVES = ["camera"]                              # Camera-based reference frame
TIME_RANGE = [8, 15]                                  # Time window for analysis in seconds
RAW = False                                           # Use processed direction data
SHOW_PLOT = False                                     # Control real-time plot display

# Suppress pandas chained assignment warnings
pd.options.mode.chained_assignment = None

# Process each recording session and perspective
for DATA_RECORDING_SESSION in DATA_RECORDING_SESSIONS:
    for PERSPECTIVE in PERSPECTIVES:
        # Load and preprocess data
        data_dir, labels_dir = data_path(DATA_RECORDING_SESSION)
        _, camera_features, camera_labels = generate_data(
            data_dir, 
            labels_dir, 
            sort=True, 
            perspective=PERSPECTIVE, 
            raw=RAW, 
            time_range=TIME_RANGE
        )

        # Define columns to exclude from analysis
        dropped_cols = ['_z', 'Position', 'delta_time', 'Combined']
        figs_healthy = []
        figs_unhealthy = []

        # Process each patient's data
        for idx in range(len(camera_features)):    
            PATIENT_NUMBER = idx

            # Only process healthy (0) and left nystagmus (3) cases
            if not (camera_labels[PATIENT_NUMBER] in [0, 3]):
                continue

            # Clean and prepare data
            df = camera_features[PATIENT_NUMBER]
            for col in dropped_cols:
                df = colsToDrop(df, col)

            # Calculate combined eye direction
            df_new = pd.DataFrame()

            if PERSPECTIVE == "world":
                df_new['abs_x_direction'] = (
                    df['worldLeftEyeDirection_x'] + df['worldRightEyeDirection_x']
                )

            if PERSPECTIVE == "camera":
                df_new['abs_x_direction'] = (
                    df['cameraLeftEyeDirection_x'] + df['cameraRightEyeDirection_x']
                )

            # Add time and center direction around zero
            df_new['time'] = df['time']
            df_new['abs_x_direction'] -= df_new['abs_x_direction'].mean()
            df = df_new

            # Generate plot
            fig = plot_sample(df, camera_labels[PATIENT_NUMBER], PATIENT_NUMBER, 
                            DATA_RECORDING_SESSION, SHOW_PLOT)

            # Update trace names and appearance
            name_mapping = {
                'abs_x_direction': 'Horizontal eye direction '
            }
            for trace in fig.data:
                if trace.name in name_mapping:
                    trace.name = name_mapping[trace.name]
                    trace.line = dict(color='purple')

            # Configure plot axes
            fig.update_yaxes(
                range=[-0.15, 0.15], 
                title_text='Elongation ratio [unitless]'
            )
            fig.update_xaxes(title_text='Time [seconds]')

            # Set title based on patient type and gaze direction
            if camera_labels[PATIENT_NUMBER] == 0:
                title = "Assessing Nystagmus in a healthy subject"
                figs_healthy.append(fig)
            elif camera_labels[PATIENT_NUMBER] == 3:
                title = "Nystagmus in a patient with left vestibular neuritis"
                
                if DATA_RECORDING_SESSION == "links":
                    title = "Nystagmus in a patient with right vestibular neuritis (left gaze)"
                elif DATA_RECORDING_SESSION == "rechts":
                    title = "Nystagmus in a patient with right vestibular neuritis (right gaze)"

                figs_unhealthy.append(fig)
                
            # Configure plot layout
            fig.update_layout(
                title_text=' ',
                autosize=False,
                width=1500,
                height=500,
                font=dict(size=20),
                font_family="Times New Roman",
                legend_title="",
                margin=dict(l=100, r=50, b=100, t=100, pad=4),
                xaxis=dict(automargin=True, tickangle=45),
                yaxis=dict(automargin=True),
                legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1
                )
            )

            # Set time range
            fig.update_xaxes(range=TIME_RANGE, title_text='Time [seconds]')

            # Display plot
            fig.show()
            
            # Save plot (fast interactive version)
            fig.write_html(f"{DIRS[3]}/{title}_{DATA_RECORDING_SESSION}.html")
            
            # High resolution version with kaleido (commented out due to long processing time)
            # fig.write_image(f"{DIRS[3]}/{title}_{DATA_RECORDING_SESSION}.svg", engine="kaleido", scale=3)

# NystagmusDownbeating vs healthy, 5 vs 0
- Labels are: {'Healthy': 0, 'Skew': 1, 'Saccades': 2, 'NystagmusLeft': 3, 'NystagmusRight': 4, 'NystagmusDownbeating': 5}

In [ ]:
"""
Analysis script for Downbeat Nystagmus detection comparing healthy subjects vs patients with downbeat syndrome.
Analyzes vertical eye movements during still position and straight gaze.
"""

# Configuration settings for data analysis
DATA_RECORDING_SESSIONS = ["still", "nase"]  # Still position and straight gaze
PERSPECTIVES = ["camera"]                     # Camera-based reference frame
TIME_RANGE = [5, 14]                         # Time window for analysis in seconds
RAW = False                                  # Use processed direction data
SHOW_PLOT = False                            # Control real-time plot display

# Suppress pandas chained assignment warnings
pd.options.mode.chained_assignment = None

# Process each recording session and perspective
for DATA_RECORDING_SESSION in DATA_RECORDING_SESSIONS:
    for PERSPECTIVE in PERSPECTIVES:
        # Load and preprocess data
        data_dir, labels_dir = data_path(DATA_RECORDING_SESSION)
        _, camera_features, camera_labels = generate_data(
            data_dir, 
            labels_dir, 
            sort=True, 
            perspective=PERSPECTIVE, 
            raw=RAW, 
            time_range=TIME_RANGE
        )

        # Define columns to exclude from analysis
        dropped_cols = ['_z', 'Position', 'delta_time', 'Combined']
        figs_healthy = []
        figs_unhealthy = []

        # Process each patient's data
        for idx in range(len(camera_features)):    
            PATIENT_NUMBER = idx

            # Only process healthy (0) and downbeat nystagmus (5) cases
            if not (camera_labels[PATIENT_NUMBER] in [0, 5]):
                continue

            # Clean and prepare data
            df = camera_features[PATIENT_NUMBER]
            for col in dropped_cols:
                df = colsToDrop(df, col)

            # Calculate combined vertical eye direction
            df_new = pd.DataFrame()
            df_new['abs_y_direction'] = (
                df['cameraLeftEyeDirection_y'] + df['cameraRightEyeDirection_y']
            )
            df_new['time'] = df['time']

            # Center vertical direction around zero
            df_new['abs_y_direction'] -= df_new['abs_y_direction'].mean()
            df = df_new

            # Generate plot
            fig = plot_sample(df, camera_labels[PATIENT_NUMBER], PATIENT_NUMBER, 
                            DATA_RECORDING_SESSION, SHOW_PLOT)

            # Update trace names and appearance
            name_mapping = {
                'abs_y_direction': 'Vertical eye direction '
            }
            for trace in fig.data:
                if trace.name in name_mapping:
                    trace.name = name_mapping[trace.name]
                    trace.line = dict(color='purple')

            # Configure plot axes
            fig.update_yaxes(
                range=[-0.15, 0.15], 
                title_text='Elongation ratio [unitless]'
            )
            fig.update_xaxes(title_text='Time [seconds]')

            # Set title based on patient type
            if camera_labels[PATIENT_NUMBER] == 0:
                title = "Assessing Nystagmus in a healthy subject"
                figs_healthy.append(fig)
            elif camera_labels[PATIENT_NUMBER] == 5:
                title = "Patient with Downbeat Nystagmus Syndrome"
                figs_unhealthy.append(fig)
                
            # Configure plot layout
            fig.update_layout(
                title_text=' ',
                autosize=False,
                width=1500,
                height=500,
                font=dict(size=20),
                font_family="Times New Roman",
                legend_title="",
                margin=dict(l=100, r=50, b=100, t=100, pad=4),
                xaxis=dict(automargin=True, tickangle=45),
                yaxis=dict(automargin=True),
                legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1
                )
            )

            # Set time range
            fig.update_xaxes(range=TIME_RANGE, title_text='Time [seconds]')

            # Display plot
            fig.show()
            
            # Save plot (fast interactive version)
            fig.write_html(f"{DIRS[4]}/{title}_{DATA_RECORDING_SESSION}.html")
            
            # High resolution version with kaleido (commented out due to long processing time)
            # fig.write_image(f"{DIRS[4]}/{title}_{DATA_RECORDING_SESSION}.svg", engine="kaleido", scale=3)